### Expressivity

In [ ]:
import json
from tqdm import tqdm
from ansatz import Ansatz
from utils import find_original_param, get_subgroup_unitaries, pennylane_to_qiskit

from qleet.analyzers.expressibility import Expressibility
from qleet.interface.circuit import CircuitDescriptor

from twirler.symmetry_groups import create_symmetric_group
from twirler.generators import get_ansatz_generators
from twirler.twirling import apply_twirling_to_generators

from qiskit import QuantumCircuit
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp

final_results = {}
n_qubits = 4

S = create_symmetric_group(n_qubits)

with open(f"groups/subgroups_{n_qubits}.json", "r") as f:
    subgroups = json.load(f)

subgroup_unitaries = get_subgroup_unitaries(subgroups, S)

for depth in range(1, 6):
    print("Depth:", depth)
    final_results[str(depth)] = {}
    for ansatz_id in range(1, 20):
        print("  Ansatz ID:", ansatz_id)
        final_results[str(depth)][str(ansatz_id)] = {"original": None, "twirled": {}}
        super_ansatz = Ansatz(ansatz_id, n_qubits, depth)
        circuit, params = super_ansatz.get_QNode()
        qiskit_circuit = pennylane_to_qiskit(circuit, n_qubits, params=params)
        qiskit_circuit.remove_final_measurements()
        params = qiskit_circuit.parameters
        circuit_descriptor = CircuitDescriptor(qiskit_circuit, params)
        exp = Expressibility(circuit_descriptor, samples=5000)
        original_expressibility = exp.expressibility()
        print(f"    Original expressibility = {original_expressibility:.4f}")
        final_results[str(depth)][str(ansatz_id)]["original"] = original_expressibility
        ansatz_generators = get_ansatz_generators(super_ansatz.get_ansatz())

        for k in tqdm(subgroup_unitaries, desc=f"n_qubits={n_qubits}, depth={depth}, ansatz={ansatz_id}"):
            final_results[str(depth)][str(ansatz_id)]["twirled"][str(k)] = []
            for elem in subgroup_unitaries[k]:
                unitaries = elem["unitaries"]
                twirled_generators = apply_twirling_to_generators(unitaries, ansatz_generators, n_qubits)
                twirled_circuit = QuantumCircuit(n_qubits)
                for i, (gen_matrix, op_wires, op_name, theta, parametrized) in enumerate(ansatz_generators):
                        twirled_elem = twirled_generators[i]
                        if twirled_elem["gate_name"] == op_name and twirled_elem["wires"] == op_wires:
                            H = twirled_elem['averaged']
                            param = find_original_param(qiskit_circuit, ansatz_generators, i, op_name, op_wires, theta)
                            pauli_op = SparsePauliOp.from_operator(H)
                            evo_gate = PauliEvolutionGate(pauli_op, time=param)
                            twirled_circuit.append(evo_gate, range(n_qubits))
                        else:
                            raise ValueError(f"Twirled generator for {op_name} on wires {op_wires} not found when {twirled_elem['gate_name']} and {twirled_elem['wires']}")
                    
                twirled_circuit.remove_final_measurements(inplace=True)
                params = twirled_circuit.parameters

                assert len(params) == len(qiskit_circuit.parameters), "Parameter count mismatch after twirling"

                circuit_descriptor = CircuitDescriptor(twirled_circuit, params)
                exp = Expressibility(circuit_descriptor, samples=5000)
                twirled_expressibility = exp.expressibility()
                print(f"    Subgroup {k}: expressibility = {twirled_expressibility:.4f}")
                final_results[str(depth)][str(ansatz_id)]["twirled"][str(k)].append(twirled_expressibility)

with open(f"results/expressibility_results_{n_qubits}.json", "w") as f:
    json.dump(final_results, f, indent=4)

Depth: 1
  Ansatz ID: 1
    Original expressibility = 0.3028


n_qubits=4, depth=1, ansatz=1:  12%|█▎        | 1/8 [00:57<06:42, 57.47s/it]

    Subgroup 1: expressibility = 0.2959
    Subgroup 2: expressibility = 1.1200
    Subgroup 2: expressibility = 1.1099
    Subgroup 2: expressibility = 1.0997
    Subgroup 2: expressibility = 0.4795
    Subgroup 2: expressibility = 0.4928
    Subgroup 2: expressibility = 0.4502
    Subgroup 2: expressibility = 0.4643
    Subgroup 2: expressibility = 0.4693


n_qubits=4, depth=1, ansatz=1:  25%|██▌       | 2/8 [10:31<36:07, 361.20s/it]

    Subgroup 2: expressibility = 0.4672
    Subgroup 3: expressibility = 1.6995
    Subgroup 3: expressibility = 1.5021
    Subgroup 3: expressibility = 1.5768


...

    Subgroup 8: expressibility = 1.0822


n_qubits=4, depth=2, ansatz=16:  88%|████████▊ | 7/8 [40:42<04:40, 280.89s/it]

    Subgroup 12: expressibility = 1.5140


n_qubits=4, depth=2, ansatz=16: 100%|██████████| 8/8 [42:13<00:00, 316.64s/it]

    Subgroup 24: expressibility = 1.6156
  Ansatz ID: 17
